In [ ]:
import numpy as np#you usually need numpy

#---these are for plots---#
import matplotlib
matplotlib.use('nbAgg')
import matplotlib.pyplot as plt

plt.rcParams['font.family']='serif'
plt.rcParams['font.size']=14
plt.rcParams['mathtext.fontset']='stix'
#-------------------------#

In [ ]:
#load the module

from os import  getcwd, chdir

curDir=getcwd()#this is the directory of the notebook
chdir('../../src')#go to the direcory that interfacePy is located
from interfacePy.Axion import Axion 
from interfacePy.WKB import relic 
from interfacePy.Cosmo import Hubble,s
from interfacePy.AxionMass import ma2

chdir(curDir)#return to the directory of the notebook

In [ ]:
#load theta_i, fa, Tosc, theta_osc, relic

#for RD universe
RDdat=np.loadtxt('RD.dat')
#for NSC
NSCdat=np.loadtxt('NSC.dat')

In [ ]:
def NSC(fa):
    '''find the approximate entropy injection factor between Tosc and today in NSC and Tosc'''
    inputFile="../../Examples/InputExamples/NSCinput.dat" 
    _=np.loadtxt(inputFile)
    for u,T,logH in _[_[:,1]<20]:
        if ma2(T,fa)**0.5>=3*np.exp(logH):
            break
    uosc,Tosc=u,T
    
    return s(_[-1][1])/s(Tosc)*np.exp(3*(_[-1][0]-uosc)),Tosc

In [ ]:
diff_i=[]
diff_osc=[]

for theta_i,fa,theta_osc,T_osc,Omegah2 in NSCdat[:10]:
    gamma,Tosc=NSC(fa)
    diff_i.append(np.abs(Omegah2-relic(T_osc,theta_i,gamma))/Omegah2)
    diff_osc.append(np.abs(Omegah2-relic(T_osc,theta_osc,gamma))/Omegah2)

    
diff_i=np.array(diff_i)
diff_osc=np.array(diff_osc)

In [ ]:
fig=plt.figure(figsize=(9,4))
fig.subplots_adjust(bottom=0.15, left=0.15, top = 0.95, right=0.9,wspace=0.0,hspace=0.0)
fig.suptitle('')

sub = fig.add_subplot(1,1,1)

sub.plot(NSCdat[:,1],diff_i,linestyle='--',c='xkcd:black',label=r"$\theta_i = \theta_{\rm osc}$")
sub.plot(NSCdat[:,1],diff_osc,linestyle='--',c='xkcd:red',label=r"Numeric $\theta_{\rm osc}$")

sub.set_xlabel(r'$f_a ~ [{\rm GeV}]$')
sub.set_ylabel(r'$\dfrac{\left|\Omega h^2 - \left(\Omega h^2\right)_{\rm WKB}\right|}{\Omega h^2}$')

sub.legend(bbox_to_anchor=(0.01, 0.9),borderaxespad=0., 
           borderpad=0.05,ncol=1,loc='upper left',fontsize=14,framealpha=0)
sub.set_yscale('log')
sub.set_xscale('log')

plt.savefig('NSC_WKB_diff.pdf',bbox_inches='tight')

fig.show()

In [ ]:
diff_i=[]
diff_osc=[]

for theta_i,fa,theta_osc,T_osc,Omegah2 in RDdat:
    diff_i.append(np.abs(Omegah2-relic(T_osc,theta_i))/Omegah2)
    diff_osc.append(np.abs(Omegah2-relic(T_osc,theta_osc))/Omegah2)

    
diff_i=np.array(diff_i)
diff_osc=np.array(diff_osc)

In [ ]:
fig=plt.figure(figsize=(9,4))
fig.subplots_adjust(bottom=0.15, left=0.15, top = 0.95, right=0.9,wspace=0.0,hspace=0.0)
fig.suptitle('')

sub = fig.add_subplot(1,1,1)

sub.plot(NSCdat[:,1],diff_i,linestyle='--',c='xkcd:black',label=r"$\theta_i = \theta_{\rm osc}$")
sub.plot(NSCdat[:,1],diff_osc,linestyle='--',c='xkcd:red',label=r"Numeric $\theta_{\rm osc}$")

sub.set_xlabel(r'$f_a ~ [{\rm GeV}]$')
sub.set_ylabel(r'$\dfrac{\left|\Omega h^2 - \left(\Omega h^2\right)_{\rm WKB}\right|}{\Omega h^2}$')

sub.legend(bbox_to_anchor=(0.01, 0.01),borderaxespad=0., 
           borderpad=0.05,ncol=1,loc='lower left',fontsize=14,framealpha=0)
sub.set_yscale('log')
sub.set_xscale('log')


plt.savefig('RD_WKB_diff.pdf',bbox_inches='tight')
fig.show()